In [1]:
import os

os.environ["DGLBACKEND"] = "pytorch"  # use PyTorch as backend
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [3]:
g = dataset[0]
g.ndata["feat"].shape

torch.Size([2708, 1433])

Below are three different models

In [4]:
from dgl.nn import GraphConv


class GCN(nn.Module):  # nn.Module - PyTorch parent class for registering all layers, parameters, etc. inside the backing
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)  # a graph convolution layer in which we specify the dimension of incoming and outgoing elements. In our case, this is a vector of words
        self.conv2 = GraphConv(h_feats, num_classes)  # the last layer, at the output we get dimension = 7 - the number of classes in the graph

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# Create the model with given dimensions
# 16 - number of parameters on the middle layer
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
model

GCN(
  (conv1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (conv2): GraphConv(in=16, out=7, normalization=both, activation=None)
)

In [5]:
class APPNP(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout=0.2, layer=2):
        super(APPNP, self).__init__()
        self.gc1 = GraphConv(nfeat, nhid) # n_feature: C, n_hidden: H
        self.gc_layer = GraphConv(nhid, nhid)  # n_hidden: H, n_hidden: H
        self.gc2 = GraphConv(nhid, nclass) # n_hidden: H, n_classes: F
        self.dropout = dropout
        self.layer = layer

    def forward(self, g, in_feat): # X, A
        x = self.gc1(g, in_feat)
        x = F.relu(x) # for APPNP paper
        for i in range(self.layer - 2):
            x = self.gc_layer(g, x)
            x = F.relu(x)  # middle conv
            x = F.dropout(x, self.dropout)
        if self.layer > 1:
            x = self.gc2(g, x) # 2th conv
        return F.log_softmax(x) # N * F

model = APPNP(g.ndata["feat"].shape[1], 20, dataset.num_classes)
model

APPNP(
  (gc1): GraphConv(in=1433, out=20, normalization=both, activation=None)
  (gc_layer): GraphConv(in=20, out=20, normalization=both, activation=None)
  (gc2): GraphConv(in=20, out=7, normalization=both, activation=None)
)

In [6]:
class CRD_CLS(nn.Module):
    def __init__(self, num_features, hidden, num_classes, dropout=0.2):
        super(CRD_CLS, self).__init__()
        self.conv1 = GraphConv(num_features, hidden)
        self.p = dropout
        self.conv2 = GraphConv(hidden, num_classes)

    def forward(self, g, in_feat):
        x = self.conv1(g, in_feat)
        x = F.relu(x)
        x = F.dropout(x, p=self.p)
        x = self.conv2(g, x)
        x = F.log_softmax(x)
        return x


model = CRD_CLS(g.ndata["feat"].shape[1], 20, dataset.num_classes)
model

CRD_CLS(
  (conv1): GraphConv(in=1433, out=20, normalization=both, activation=None)
  (conv2): GraphConv(in=20, out=7, normalization=both, activation=None)
)

#### Loop for model training

In [7]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )


train(g, model)

In epoch 0, loss: 1.946, val acc: 0.224 (best 0.224), test acc: 0.210 (best 0.210)


C:\Users\Atikin\AppData\Local\Temp/ipykernel_9164/165596389.py:13: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


In epoch 5, loss: 1.888, val acc: 0.498 (best 0.498), test acc: 0.546 (best 0.546)


In epoch 10, loss: 1.798, val acc: 0.600 (best 0.626), test acc: 0.648 (best 0.623)
In epoch 15, loss: 1.674, val acc: 0.654 (best 0.698), test acc: 0.677 (best 0.679)
In epoch 20, loss: 1.529, val acc: 0.676 (best 0.698), test acc: 0.689 (best 0.679)
In epoch 25, loss: 1.363, val acc: 0.684 (best 0.712), test acc: 0.721 (best 0.710)
In epoch 30, loss: 1.199, val acc: 0.706 (best 0.720), test acc: 0.714 (best 0.712)
In epoch 35, loss: 1.014, val acc: 0.692 (best 0.720), test acc: 0.715 (best 0.712)
In epoch 40, loss: 0.823, val acc: 0.708 (best 0.730), test acc: 0.723 (best 0.713)
In epoch 45, loss: 0.695, val acc: 0.674 (best 0.730), test acc: 0.702 (best 0.713)
In epoch 50, loss: 0.598, val acc: 0.742 (best 0.742), test acc: 0.734 (best 0.734)
In epoch 55, loss: 0.459, val acc: 0.766 (best 0.766), test acc: 0.740 (best 0.742)
In epoch 60, loss: 0.366, val acc: 0.726 (best 0.766), test acc: 0.753 (best 0.742)
In epoch 65, loss: 0.312, val acc: 0.766 (best 0.766), test acc: 0.741 (best

Let's see what accuracy we got in the end

In [8]:
model.eval()
features = g.ndata["feat"]
pred = model(g, features).argmax(dim=1)
print(pred)
correct = (pred[g.ndata['test_mask']] == g.ndata["label"][g.ndata["test_mask"]]).sum()
acc = int(correct) / int(g.ndata['test_mask'].sum())
print(f'Accuracy: {acc:.4f}')

tensor([3, 4, 4,  ..., 3, 4, 3])
Accuracy: 0.7540


C:\Users\Atikin\AppData\Local\Temp/ipykernel_9164/165596389.py:13: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


Now we save the entire model to a file. Since the model uses PyTorch as a back-up, saving occurs through PyTorch functions

In [9]:
torch.save(model, './models/crd_cls_model.pt')

#### Now let’s load the model and make an inference

In [10]:
model_inf = torch.load('./models/crd_cls_model.pt')
model_inf.eval()

CRD_CLS(
  (conv1): GraphConv(in=1433, out=20, normalization=both, activation=None)
  (conv2): GraphConv(in=20, out=7, normalization=both, activation=None)
)

In [11]:
features = g.ndata["feat"]
pred = model_inf(g, features).argmax(dim=1)
print(pred)
correct = (pred[g.ndata['test_mask']] == g.ndata["label"][g.ndata["test_mask"]]).sum()
acc = int(correct) / int(g.ndata['test_mask'].sum())
print(f'Accuracy: {acc:.4f}')

tensor([3, 4, 4,  ..., 3, 3, 3])
Accuracy: 0.7420


C:\Users\Atikin\AppData\Local\Temp/ipykernel_9164/165596389.py:13: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)
